In [ ]:
import numpy as np
import pandas as pd
from scipy import stats, linalg
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Append base directory
import os,sys #,inspect
rootname = "pub-2020-exploratory-analysis"
#thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)


from mesostat.metric.dim3d.exp_var_2nd_order import quadratic_triplet_decomp_1D

import lib.nullmodels.null3D as null3D
import lib.nullmodels.pidtest as pidtest

%load_ext autoreload
%autoreload 2

In [ ]:
funcDict = {
    'red':    null3D.gen_data_red_noisy,
    'unq_xz': null3D.gen_data_unq_noisy,
    'xor_z':  null3D.gen_data_xor_noisy
}

## Noise-dependence

### Effect sizes for example variance

In [ ]:
decompLabels = ['unq_s1', 'unq_s2', 'red', 'syn']
labeledDecomp = lambda x,y,z: dict(zip(decompLabels, quadratic_triplet_decomp_1D(x,y,z)))

In [ ]:
for funcName, func in funcDict.items():

    gen_data_eff = lambda: func(n=10000, sigX=1, sigY=1, sigZ=1)
    rezDF   = pidtest.run_tests(gen_data_eff, labeledDecomp, decompLabels, nTest=200)
    rezDFsh = pidtest.run_tests(gen_data_eff, labeledDecomp, decompLabels, nTest=200, haveShuffle=True)

    pidtest.plot_test_summary(rezDF, rezDFsh, haveEff=False, suptitle=funcName)
    plt.savefig(funcName + '_r2_summary_rand.png', dpi=200)
    plt.show()

### Effect of variance

In [ ]:
for funcName, func in funcDict.items():
    print(funcName)
    
    funcEff = lambda alpha: func(n=1000, sigX=alpha, sigY=alpha, sigZ=alpha)
    
    pidtest.run_plot_param_effect(funcEff, labeledDecomp, decompLabels, nTest=1000, alphaRange=(0, 2))
    plt.show()

In [ ]:
nSample = 10000
for funcName, func in funcDict.items():
    print(funcName)

    funcEff = lambda alpha: func(n=nSample, sigX=alpha, sigY=alpha, sigZ=alpha)
    pidtest.run_plot_param_effect_test(funcEff, labeledDecomp, decompLabels, nStep=10, nTest=400, alphaRange=(0, 2))    
    
    plt.savefig(funcName + '_r2_vareff_n'+str(nSample)+'.png', dpi=200)
    plt.show()

### Effect of number of samples

In [ ]:
sig = 1.0
for funcName, func in funcDict.items():
    print(funcName)
    
    funcEff = lambda n: func(n=n, sigX=sig, sigY=sig, sigZ=sig)
    pidtest.run_plot_data_effect_test(funcEff, labeledDecomp, decompLabels, nStep=10, nTest=400)
    plt.savefig(funcName + '_r2_nEff_sig'+str(sig)+'.png', dpi=200)
    plt.show()

### Synergy-Redundancy Relationship

In [ ]:
for funcName, func in funcDict.items():
    print(funcName)
    
#     funcEff = lambda n, a, b, c: func(n=n, sigX=1.0, sigY=1.0, sigZ=1.0)
    pidtest.run_plot_scatter(func, labeledDecomp, 'red', 'syn', 3, varLimits=(0, 2), nTestDim=20)